In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import collections
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from nltk import ne_chunk
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cnn-dailymail/cnn_dailymail-test.parquet
/kaggle/input/cnn-daily/cnn_dailymail-validation(1).parquet


# DATA ANALYSIS AND DATA PREPROCESSING

In [46]:
# Training data
data  = pd.read_parquet("/kaggle/input/cnn-dailymail/cnn_dailymail-test.parquet")

In [47]:
data.head()

,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f


In [48]:
data = data[['id' , 'article' , 'highlights']]
data.head()

,id,article,highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,230c522854991d053fe98a718b1defa077a8efef,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...


In [49]:
data.shape

(11490, 3)

In [50]:
data.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [51]:
data.duplicated().sum()

0

In [52]:
# Testing Data
data2  = pd.read_parquet("/kaggle/input/cnn-daily/cnn_dailymail-validation(1).parquet")

In [53]:
data2.head()

,article,highlights,id
0,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...,a4942dd663020ca54575471657a0af38d82897d6
1,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend . Leag...,4157bc4da185971e2742f349d69a037343bc0d95
2,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...,60736693e3b1b32d14337a317190c6606e879a85
3,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...,8cdf9cc3ed0276b7a7944cc18ba459355b5984ad
4,(CNN)A Pennsylvania community is pulling toget...,"Cayman Naib, 13, hasn't been heard from since ...",2e6613d531843515bf5401286cc3e45c4df530d2


In [54]:
data2 = data2[['id' , 'article' , 'highlights']]
data2.head()

,id,article,highlights
0,a4942dd663020ca54575471657a0af38d82897d6,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...
1,4157bc4da185971e2742f349d69a037343bc0d95,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend . Leag...
2,60736693e3b1b32d14337a317190c6606e879a85,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...
3,8cdf9cc3ed0276b7a7944cc18ba459355b5984ad,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...
4,2e6613d531843515bf5401286cc3e45c4df530d2,(CNN)A Pennsylvania community is pulling toget...,"Cayman Naib, 13, hasn't been heard from since ..."


In [55]:
data.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [56]:
data.duplicated().sum()

0

In [57]:
merged_data = pd.concat([data, data2], ignore_index=True)

In [58]:
merged_data.head()

,id,article,highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,230c522854991d053fe98a718b1defa077a8efef,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...


In [59]:
merged_data.shape

(24858, 3)

In [60]:
text_length_data = merged_data[['id' , 'article' , 'highlights']]

## Text Length Analysis

In [61]:
text_length_data["length_article"] = merged_data["article"].str.len()
text_length_data["length_highlights"] = merged_data["highlights"].str.len()
text_length_data.head()

,id,article,highlights,length_article,length_highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,3612,233
1,230c522854991d053fe98a718b1defa077a8efef,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",2331,214
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4128,218
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,1191,254
4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,2129,260


## Word Frequency Analysis
### using tokenization

In [62]:
articles = merged_data["article"]
# Combine all articles into a single string
text = " ".join(articles)
tokens = nltk.word_tokenize(text)

# Remove stopwords
stop_words = set(stopwords.words("english"))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

#remove punctuations
filtered_tokens = [word for word in filtered_tokens if word not in string.punctuation]
word_freq = Counter(filtered_tokens)
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

#most common words in the column
top_n = 10
for word, freq in sorted_word_freq[:top_n]:
    print(f"{word}: {freq}")


's: 143144
said: 92102
’: 83362
‘: 37903
one: 33758
n't: 32307
would: 32098
also: 30602
last: 26063
two: 25275


In [63]:
highli = merged_data["highlights"]
# Combine all highlights into a single string
text = " ".join(highli)
tokens = nltk.word_tokenize(text)

# Remove stopwords
stop_words = set(stopwords.words("english"))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

#remove punctuations
filtered_tokens = [word for word in filtered_tokens if word not in string.punctuation]
word_freq = Counter(filtered_tokens)
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

#most common words in the column
top_n = 10
for word1, freq1 in sorted_word_freq[:top_n]:
    print(f"{word1}: {freq1}")

's: 12386
said: 4595
says: 2897
last: 2391
two: 2320
also: 2294
years: 2029
found: 2023
one: 1966
first: 1866


## Named Entity Recognition

In [64]:
articles = merged_data["article"]
entity_types = ['PERSON', 'ORGANIZATION']
for article in articles:
    sentences = nltk.sent_tokenize(article)
    count = 0
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged_words = nltk.pos_tag(words)
        named_entities = ne_chunk(tagged_words)
        for entity in named_entities:
            if hasattr(entity, 'label') and entity.label() in entity_types:
                print(entity.label(), ' '.join(c[0] for c in entity.leaves()))
                count += 1
                if count == 10:
                    break
        if count == 10:
            break

ORGANIZATION CNN
ORGANIZATION International Criminal Court
ORGANIZATION ICC
PERSON East Jerusalem
ORGANIZATION ICC
ORGANIZATION ICC
PERSON Riad
ORGANIZATION State Party
ORGANIZATION Rome Statute
ORGANIZATION ICC
ORGANIZATION CNN
ORGANIZATION Veterinary
PERSON Moses Lake
PERSON Sara Mellado
PERSON Mellado
ORGANIZATION WSU News
ORGANIZATION Good Samaritan Fund
PERSON Mellado
PERSON Theia
PERSON Mellado
ORGANIZATION CNN
PERSON Mohammad Javad Zarif
ORGANIZATION State
PERSON John Kerry
PERSON Mohammad Javad Zarif
PERSON Zarif
PERSON Zarif
PERSON Happy Rosh Hashanah
PERSON Christine Pelosi
ORGANIZATION House
ORGANIZATION CNN
ORGANIZATION Nebraska Medicine
PERSON Taylor Wilson
ORGANIZATION Partners
ORGANIZATION National Institutes
ORGANIZATION Centers
PERSON Disease Control
ORGANIZATION World
PERSON Sierra Leone
ORGANIZATION CNN
ORGANIZATION Duke
PERSON Duke
PERSON Richard Brodhead
ORGANIZATION University
ORGANIZATION University
PERSON Duke
PERSON Larry Moneta
PERSON Duke
ORGANIZATION Univers

KeyboardInterrupt: 

In [65]:
merged_data.head()

,id,article,highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,230c522854991d053fe98a718b1defa077a8efef,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...


### Remove Stop Words and Tokenizing

In [66]:
articles = merged_data["article"]
stop_words = set(stopwords.words("english"))
filtered_articles = []
for article in articles:
    words = nltk.word_tokenize(article)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_article = " ".join(filtered_words)
    filtered_articles.append(filtered_article)
merged_data["article"] = filtered_articles

In [67]:
merged_data['article'].head()

0    ( CNN ) Palestinian Authority officially becam...
1    ( CNN ) Never mind cats nine lives . stray poo...
2    ( CNN ) 've following news lately , certain th...
3    ( CNN ) Five Americans monitored three weeks O...
4    ( CNN ) Duke student admitted hanging noose ma...
Name: article, dtype: object

In [68]:
highli = merged_data["highlights"]
stop_words = set(stopwords.words("english"))
filtered_highli = []
for x in highli:
    words = nltk.word_tokenize(x)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_highlig = " ".join(filtered_words)
    filtered_highli.append(filtered_highlig)
merged_data["highlights"] = filtered_highli

In [69]:
merged_data['highlights'].head()

0    Membership gives ICC jurisdiction alleged crim...
1    Theia , bully breed mix , apparently hit car ,...
2    Mohammad Javad Zarif spent time John Kerry for...
3    17 Americans exposed Ebola virus Sierra Leone ...
4    Student longer Duke University campus face dis...
Name: highlights, dtype: object

In [70]:
# data after removing stop words
merged_data.head()

,id,article,highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,( CNN ) Palestinian Authority officially becam...,Membership gives ICC jurisdiction alleged crim...
1,230c522854991d053fe98a718b1defa077a8efef,( CNN ) Never mind cats nine lives . stray poo...,"Theia , bully breed mix , apparently hit car ,..."
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"( CNN ) 've following news lately , certain th...",Mohammad Javad Zarif spent time John Kerry for...
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,( CNN ) Five Americans monitored three weeks O...,17 Americans exposed Ebola virus Sierra Leone ...
4,c27cf1b136cc270023de959e7ab24638021bc43f,( CNN ) Duke student admitted hanging noose ma...,Student longer Duke University campus face dis...


## Stemming

In [71]:
column1 = merged_data["article"]
column2 = merged_data["highlights"]
stemmer = PorterStemmer()
stemmed_column1 = []
stemmed_column2 = []
# Apply stemming to each column
for row in column1:
    stemmed_text = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(row)])
    stemmed_column1.append(stemmed_text)

for row in column2:
    stemmed_text = ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(row)])
    stemmed_column2.append(stemmed_text)

merged_data["article"] = stemmed_column1
merged_data["highlights"] = stemmed_column2


In [72]:
merged_data.head()

,id,article,highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,( cnn ) palestinian author offici becam 123rd ...,membership give icc jurisdict alleg crime comm...
1,230c522854991d053fe98a718b1defa077a8efef,( cnn ) never mind cat nine live . stray pooch...,"theia , bulli breed mix , appar hit car , whac..."
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"( cnn ) 've follow news late , certain thing d...",mohammad javad zarif spent time john kerri for...
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,( cnn ) five american monitor three week omaha...,17 american expos ebola viru sierra leon march...
4,c27cf1b136cc270023de959e7ab24638021bc43f,( cnn ) duke student admit hang noos made rope...,student longer duke univers campu face discipl...
